You are investigating various trade-oﬀs in the design of an autonomous blimp. The system dynamics are given as below, with the system state $x(k)\in{\mathbb{R}^2}$ comprising the height ($x_1$ in units of m) and vertical velocity ($x_2$ in units of m/s) of the blimp, driven by a random acceleration: 
$$
x(k)=\begin{bmatrix} x_1(k) \\ x_2(k) \end{bmatrix}=\begin{bmatrix} 1 & \Delta t \\ 0 & 1 \end{bmatrix}x(k-1)+v(k-1)
$$  
where $\Delta t$ is the sampling time, and $v(k-1)$ is zero-mean white noise, with variance parametrized through the scalar $\sigma _ a ^ 2$:  
$$
Var[v(k-1)]={\sigma _ a ^ 2}\begin{bmatrix} \frac{1}{4}{\Delta t^4} & \frac{1}{2}{\Delta t^3} \\ \frac{1}{2}{\Delta t^3} & \Delta t^2 \end{bmatrix}
$$  
The original system is equipped with a GPS sensor, so that:
$$
z_1(k)=x_1(k)+w_1(k)
$$  
where $w_1(k)$ is a zero-mean, white noise sequence with variance $Var[w_1(k)]=m_1^2$. We are given $\Delta t=\frac{1}{10}s$, $\sigma_a=5m/s^2$, and $m_1=10m$.  

The robot's mission relies on accurate knowledge of its position, which we will quantify with the metric $J_p:=\sqrt{\lim_{k \rightarrow \infty} Var[x_1(k)|z(1:k)]}$. 

We first compute $J_p$ as a function of the usual Kalman ﬁlter matrices $A, \Sigma_{vv}, H$ and $\Sigma_{ww}$.

In [1]:
import numpy as np
from scipy import linalg
from IPython.display import Latex

deltat = 1/10
sigma_a = 5
m1 = 10

A = np.array([[1, deltat],
              [0, 1]])
H = np.array([[1, 0]])
Sigma_vv = sigma_a ** 2 * np.array([[deltat ** 4 / 4, deltat ** 3 / 2],
                                   [deltat ** 3 / 2, deltat ** 2]])
Sigma_ww = np.array([[m1 ** 2]])

P_inf = linalg.solve_discrete_are(A.T, H.T, Sigma_vv, Sigma_ww)
Pm_inf = P_inf - P_inf @ H.T @ linalg.inv(H @ P_inf @ H.T + Sigma_ww) @ H @ P_inf
Jp = np.sqrt(Pm_inf[0, 0])

Latex(r'$J_p=%.3f.$'%(Jp))

<IPython.core.display.Latex object>

We are considering various ways to improve the system’s estimation performance, enumerated below. Each modiﬁcation is an independent modiﬁcation of the original system. For each of the following suggested modiﬁcations to the system, we compute the performance metric $J_p$.

1) Replace the GPS sensor with that of brand A, which has $m_1=5m$ (the noise standard deviation is cut in half).

In [2]:
import numpy as np
from scipy import linalg
from IPython.display import Latex

deltat = 1/10
sigma_a = 5
m1 = 5

A = np.array([[1, deltat],
              [0, 1]])
H = np.array([[1, 0]])
Sigma_vv = sigma_a ** 2 * np.array([[deltat ** 4 / 4, deltat ** 3 / 2],
                                   [deltat ** 3 / 2, deltat ** 2]])
Sigma_ww = np.array([[m1 ** 2]])

P_inf = linalg.solve_discrete_are(A.T, H.T, Sigma_vv, Sigma_ww)
Pm_inf = P_inf - P_inf @ H.T @ linalg.inv(H @ P_inf @ H.T + Sigma_ww) @ H @ P_inf
Jp = np.sqrt(Pm_inf[0, 0])

Latex(r'$J_p=%.3f.$'%(Jp))

<IPython.core.display.Latex object>

2) Replace the GPS sensor with that of brand B, which has $\Delta t=\frac{1}{20}s$, and $m_1=10m$ (i.e. the sensor returns equally noisy measurements, but twice as frequently). 

In [3]:
import numpy as np
from scipy import linalg
from IPython.display import Latex

deltat = 1/20
sigma_a = 5
m1 = 10

A = np.array([[1, deltat],
              [0, 1]])
H = np.array([[1, 0]])
Sigma_vv = sigma_a ** 2 * np.array([[deltat ** 4 / 4, deltat ** 3 / 2],
                                   [deltat ** 3 / 2, deltat ** 2]])
Sigma_ww = np.array([[m1 ** 2]])

P_inf = linalg.solve_discrete_are(A.T, H.T, Sigma_vv, Sigma_ww)
Pm_inf = P_inf - P_inf @ H.T @ linalg.inv(H @ P_inf @ H.T + Sigma_ww) @ H @ P_inf
Jp = np.sqrt(Pm_inf[0, 0])

Latex(r'$J_p=%.3f.$'%(Jp))

<IPython.core.display.Latex object>

3) Retain the original GPS sensor, and add an airspeed sensor which gives the additional measurement $z_2(k)=x_2(k)+w_2(k)$, where $w_2(k)$ is a zero-mean, white noise sequence, independent of all quantities, and with $Var[w_2(k)]=m_2^2$ with $m_2=1m/s$. The sensors $z_1$ and $z_2$ return data at the same instants of time.

In [4]:
import numpy as np
from scipy import linalg
from IPython.display import Latex

deltat = 1/10
sigma_a = 5
m1 = 10
m2 = 1

A = np.array([[1, deltat],
              [0, 1]])
H = np.array([[1, 0],
              [0, 1]])
Sigma_vv = sigma_a ** 2 * np.array([[deltat ** 4 / 4, deltat ** 3 / 2],
                                   [deltat ** 3 / 2, deltat ** 2]])
Sigma_ww = np.array([[m1 ** 2, 0],
                     [0, m2 ** 2]])

P_inf = linalg.solve_discrete_are(A.T, H.T, Sigma_vv, Sigma_ww)
Pm_inf = P_inf - P_inf @ H.T @ linalg.inv(H @ P_inf @ H.T + Sigma_ww) @ H @ P_inf
Jp = np.sqrt(Pm_inf[0, 0])

Latex(r'$J_p=%.3f.$'%(Jp))

<IPython.core.display.Latex object>

4) Retain the original GPS sensor, and add a second, independent barometric height sensor $z_2(k)=x_1(k)+w_2(k)$, where $w_2(k)$ is a zero-mean, white noise sequence, independent of all quantities, and with $Var[w_2(k)]=m_2^2$ with $m_2=10m$. The sensors $z_1$ and $z_2$ return data at the same instants of time.

In [5]:
import numpy as np
from scipy import linalg
from IPython.display import Latex

deltat = 1/10
sigma_a = 5
m1 = 10
m2 = 10

A = np.array([[1, deltat],
              [0, 1]])
H = np.array([[1, 0],
              [1, 0]])
Sigma_vv = sigma_a ** 2 * np.array([[deltat ** 4 / 4, deltat ** 3 / 2],
                                   [deltat ** 3 / 2, deltat ** 2]])
Sigma_ww = np.array([[m1 ** 2, 0],
                     [0, m2 ** 2]])

P_inf = linalg.solve_discrete_are(A.T, H.T, Sigma_vv, Sigma_ww)
Pm_inf = P_inf - P_inf @ H.T @ linalg.inv(H @ P_inf @ H.T + Sigma_ww) @ H @ P_inf
Jp = np.sqrt(Pm_inf[0, 0])

Latex(r'$J_p=%.3f.$'%(Jp))

<IPython.core.display.Latex object>

5) Retain the original GPS sensor, and add a second identical GPS sensor from the same manufacturer, so that $z_2(k)=x_1(k)+w_2(k)$, where $w_2(k)=w_1(k)$. 

In [6]:
import numpy as np
from scipy import linalg
from IPython.display import Latex

deltat = 1/10
sigma_a = 5
m1 = 10

A = np.array([[1, deltat],
              [0, 1]])
H = np.array([[1, 0],
              [1, 0]])
Sigma_vv = sigma_a ** 2 * np.array([[deltat ** 4 / 4, deltat ** 3 / 2],
                                   [deltat ** 3 / 2, deltat ** 2]])
Sigma_ww = np.array([[m1 ** 2, m1 ** 2],
                     [m1 ** 2, m1 ** 2 + 0.01]]) # to avoid singular matrix during computing inverse

P_inf = linalg.solve_discrete_are(A.T, H.T, Sigma_vv, Sigma_ww)
Pm_inf = P_inf - P_inf @ H.T @ linalg.inv(H @ P_inf @ H.T + Sigma_ww) @ H @ P_inf
Jp = np.sqrt(Pm_inf[0, 0])

Latex(r'$J_p=%.3f.$'%(Jp))

<IPython.core.display.Latex object>

6) Retain the original sensor, but modify the system design by making it more aerodynamic and thereby reducing the eﬀect of aerodynamic disturbances, so that $\sigma_a=1m/s$. 

In [7]:
import numpy as np
from scipy import linalg
from IPython.display import Latex

deltat = 1/10
sigma_a = 1
m1 = 10

A = np.array([[1, deltat],
              [0, 1]])
H = np.array([[1, 0]])
Sigma_vv = sigma_a ** 2 * np.array([[deltat ** 4 / 4, deltat ** 3 / 2],
                                   [deltat ** 3 / 2, deltat ** 2]])
Sigma_ww = np.array([[m1 ** 2]])

P_inf = linalg.solve_discrete_are(A.T, H.T, Sigma_vv, Sigma_ww)
Pm_inf = P_inf - P_inf @ H.T @ linalg.inv(H @ P_inf @ H.T + Sigma_ww) @ H @ P_inf
Jp = np.sqrt(Pm_inf[0, 0])

Latex(r'$J_p=%.3f.$'%(Jp))

<IPython.core.display.Latex object>